<a href="https://colab.research.google.com/github/stayup24h/Hangul-to-Unicode-Obfuscation-Project/blob/main/unicode_similarity_search_by_OCR_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from sklearn.model_selection import KFold
from copy import deepcopy # 각 폴드마다 새로운 모델을 시작하기 위함

In [5]:
# autoencoder 정의

def compiled_autoencoder(input_dim=68, latent_dim=16):
    """Autoencoder 모델을 정의하고 컴파일하여 반환합니다."""

    # 1. Encoder 정의
    input_layer = tf.keras.layers.Input(shape=(input_dim,), name='input_vector')
    x = tf.keras.layers.Dense(32, activation='relu')(input_layer)
    latent_vector = tf.keras.layers.Dense(latent_dim, activation='relu', name='latent_vector')(x)
    encoder = Model(input_layer, latent_vector, name='encoder')

    # 2. Decoder 정의
    latent_input = tf.keras.layers.Input(shape=(latent_dim,), name='latent_input')
    y = tf.keras.layers.Dense(32, activation='relu')(latent_input)
    reconstruction = tf.keras.layers.Dense(input_dim, activation='sigmoid', name='reconstruction')(y)
    decoder = Model(latent_input, reconstruction, name='decoder')

    # 3. Autoencoder 정의
    autoencoder_output = decoder(encoder(input_layer))
    autoencoder = Model(input_layer, autoencoder_output, name='autoencoder')

    # 4. 컴파일
    autoencoder.compile(optimizer='adam', loss='mse')

    return autoencoder, encoder

In [ ]:
# autoencoder 학습

# TODO : 유니코드를 OCR한 68차원 numpy 배열 불러오기

N = 200000 # numpy 배열 크기
INPUT_DIM = 68
X_features_all = np.random.rand(N, INPUT_DIM).astype('float32')
LATENT_DIM = 16

K_FOLDS = 10
kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

EPOCHS = 20
BATCH_SIZE = 256

fold_results = []
trained_encoders = []
best_loss = float('inf')
best_encoder = None

print(f"Starting {K_FOLDS}-Fold Cross-Validation")

for fold, (train_index, val_index) in enumerate(kf.split(X_features_all)):
    print(f"\n--- Fold {fold+1}/{K_FOLDS} ---")

    X_train, X_val = X_features_all[train_index], X_features_all[val_index]

    autoencoder, current_encoder = compiled_autoencoder(INPUT_DIM, LATENT_DIM)

    history = autoencoder.fit(
        x=X_train,
        y=X_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, X_val),
        verbose=1
    )

    val_loss = history.history['val_loss'][-1]
    fold_results.append(val_loss)
    print(f"Fold {fold+1} Validation Loss: {val_loss:.6f}")

    if val_loss < best_loss:
        best_loss = val_loss
        best_encoder = deepcopy(current_encoder)

mean_val_loss = np.mean(fold_results)
std_val_loss = np.std(fold_results)

print("\n--- K-Fold Summary ---")
print(f"Individual Fold Losses: {fold_results}")
print(f"Mean Validation Loss across {K_FOLDS} Folds: {mean_val_loss:.6f} (+/- {std_val_loss:.6f})")

if best_encoder:
    best_encoder.save('best_kf_encoder_for_faiss.h5')
    print("\nBest Encoder Model saved successfully!")

In [ ]:
# 68차원 벡터들을 16차원 벡터들로 변환

best_encoder = load_model('best_kf_encoder_for_faiss.h5')

# TODO : 유니코드를 OCR한 68차원 numpy 배열 불러오기

# X_features_all = np.load('all_unicode_features.npy')
N = 200000 #numpy 배열 크기
INPUT_DIM = 68
X_features_all = np.random.rand(N, INPUT_DIM).astype('float32') # 예시 데이터

X_16dim_vectors = best_encoder.predict(X_features_all)

X_16dim_vectors = X_16dim_vectors.astype('float32')

In [ ]:
# 변환한 벡터들 저장

VECTOR_OUTPUT_FILENAME = "faiss_16dim_vectors.npy"

np.save(VECTOR_OUTPUT_FILENAME, X_16dim_vectors)

print(f"16차원 잠재 벡터가 {VECTOR_OUTPUT_FILENAME}에 저장되었습니다. (Shape: {X_16dim_vectors.shape})")

In [ ]:
# 유니코드 코드 포인트 배열 생성 (N개의 요소)
# 이 배열은 X_features_all을 생성했을 때의 순서와 정확히 일치해야 합니다.
# 예시: 한글 외 유니코드 코드 포인트만 순서대로 나열
# [0x0000, 0x0001, ..., 0x00A0, ..., 0x10FFFF] (제외 범위 제외)
unicode_codepoints = np.array([
    # 실제 데이터셋을 만들 때 사용된 코드 포인트를 순서대로 여기에 배치해야 함.
    i for i in range(N) # 예시에서는 N개의 임의 인덱스라고 가정
])

MAPPING_OUTPUT_FILENAME = "faiss_unicode_map.npy"
np.save(MAPPING_OUTPUT_FILENAME, unicode_codepoints)

print(f"유니코드 코드 포인트 매핑이 {MAPPING_OUTPUT_FILENAME}에 저장되었습니다. (Shape: {unicode_codepoints.shape})")

In [ ]:
%pip install faiss-cpu numpy
%pip install faiss-gpu

In [ ]:
import numpy as np
import faiss

# --- 저장된 파일 이름 ---
VECTOR_OUTPUT_FILENAME = "faiss_16dim_vectors.npy"
MAPPING_OUTPUT_FILENAME = "faiss_unicode_map.npy"
QUERY_ENCODER_MODEL = "best_kf_encoder_for_faiss.h5"
FAISS_INDEX_FILENAME = "unicode_faiss_index.faiss"

# 1. 16차원 잠재 벡터 로드 (N x 16 형태)
# N은 약 20만
X_vectors = np.load(VECTOR_OUTPUT_FILENAME)
D = X_vectors.shape[1]  # D = 16 (벡터 차원)

# 2. 유니코드 매핑 정보 로드 (N개 코드 포인트)
# 이 배열의 순서가 X_vectors의 행 순서와 일치해야 합니다.
U_map = np.load(MAPPING_OUTPUT_FILENAME)

In [ ]:
# --- FAISS 인덱스 설정 ---
NLIST = 500  # 클러스터(Partition) 개수. (데이터 개수의 sqrt 근처 값 권장)

# 1. Quantizer 정의: 벡터를 클러스터링할 때 사용할 기준 (일반적으로 L2 거리를 사용하는 Flat 인덱스 사용)
quantizer = faiss.IndexFlatL2(D)

# 2. IVF 인덱스 생성: D차원, NLIST개의 클러스터, L2 거리 사용
index = faiss.IndexIVFFlat(quantizer, D, NLIST, faiss.METRIC_L2)

# 3. 인덱스 훈련 (Training)
# IVF 인덱스는 add 전에 클러스터 중심점을 계산하는 훈련이 필요합니다.
print("FAISS Index Training Start...")
index.train(X_vectors)
print("Training Complete. Index is trained:", index.is_trained)

# 4. 벡터 추가 (Adding)
# 16차원 잠재 벡터를 인덱스에 추가합니다.
index.add(X_vectors)
print(f"Total vectors added: {index.ntotal}")

# 5. 검색 속도 vs. 정확도 설정 (nprobe)
# 검색 시 확인할 클러스터의 개수. 높을수록 정확하나 느립니다.
# 20만 개에서는 10~30 사이가 적절하며, 20을 추천합니다.
index.nprobe = 20

# 6. 인덱스 저장
faiss.write_index(index, FAISS_INDEX_FILENAME)
print(f"FAISS Index saved as {FAISS_INDEX_FILENAME}")

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf

# Keras 모델 로드 (query_encoder)
try:
    # Autoencoder 학습에서 저장한 인코더 모델 로드
    query_encoder = load_model(QUERY_ENCODER_MODEL) 
except Exception as e:
    # 모델 로드 실패 시 임시 더미 함수 사용 (실제 환경에서는 반드시 로드 필요)
    print(f"경고: 인코더 모델 로드 실패. 실제 사용 시 {QUERY_ENCODER_MODEL}을 로드해야 합니다.")
    query_encoder = lambda x: np.random.rand(1, 16).astype('float32') 

# FAISS 인덱스 로드
index = faiss.read_index(FAISS_INDEX_FILENAME)

def find_similar_unicode(korean_char, k=5):
    """
    한글 문자를 입력받아 시각적으로 가장 유사한 K개의 유니코드 문자를 반환합니다.
    """
    
    # --- 1. 쿼리 벡터 생성 (OCR 예측 및 인코더 변환 필요) ---
    
    # 실제 구현 필요: 
    # 1. korean_char를 이미지로 렌더링
    # 2. OCR 모델로 예측하여 68차원 특징 벡터 (cho_vec, jung_vec, jong_vec) 생성
    # 3. 68차원 특징 벡터를 하나의 배열로 조합 (q_68dim)
    
    # 여기서는 68차원 특징 벡터가 이미 준비되었다고 가정하고 변환만 수행
    
    # 예시: 임의의 68차원 벡터 생성 (실제는 OCR 모델 출력)
    dummy_q_68dim = np.random.rand(1, 68).astype('float32') 
    
    # 쿼리 벡터를 16차원 잠재 벡터로 변환
    q_16dim = query_encoder.predict(dummy_q_68dim, verbose=0)
    q_16dim = q_16dim.astype('float32') # FAISS 요구 타입

    # --- 2. FAISS 검색 ---
    # D: 거리(Distance) 배열, I: 인덱스(Index) 배열
    # 검색된 인덱스 I는 U_map의 인덱스와 일치합니다.
    D, I = index.search(q_16dim, k) 

    # --- 3. 결과 해석 및 반환 ---
    results = []
    
    # 가장 가까운 이웃의 인덱스와 거리를 순회합니다.
    for rank, (idx, distance) in enumerate(zip(I[0], D[0])):
        if idx >= 0: # 유효한 인덱스일 경우
            code_point = U_map[idx]
            unicode_char = chr(code_point)
            
            results.append({
                "Rank": rank + 1,
                "Character": unicode_char,
                "CodePoint": f"U+{code_point:04X}",
                "Similarity_Distance": float(distance) # 거리가 낮을수록 유사
            })
            
    return results

# --- 사용 예시 ---
input_char = "한"
similar_chars = find_similar_unicode(input_char, k=10)

print(f"--- '{input_char}'와 시각적으로 유사한 상위 10개 유니코드 문자 ---")
for result in similar_chars:
    print(
        f"[{result['Rank']}] {result['Character']} "
        f"({result['CodePoint']}) - Distance: {result['Similarity_Distance']:.4f}"
    )